In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:


image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)



Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

# Problem 1

In [5]:
batch_size = 128
beta=0.01
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  # Loss function

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # regularization L2
  regularizer=tf.nn.l2_loss(weights)
  loss=tf.reduce_mean(loss +beta*regularizer)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 48.939823
Minibatch accuracy: 3.1%
Validation accuracy: 5.4%
Minibatch loss at step 500: 11.268654
Minibatch accuracy: 75.8%
Validation accuracy: 72.7%
Minibatch loss at step 1000: 4.534639
Minibatch accuracy: 76.6%
Validation accuracy: 77.4%
Minibatch loss at step 1500: 1.804006
Minibatch accuracy: 82.8%
Validation accuracy: 80.0%
Minibatch loss at step 2000: 1.055872
Minibatch accuracy: 87.5%
Validation accuracy: 81.6%
Minibatch loss at step 2500: 0.903797
Minibatch accuracy: 79.7%
Validation accuracy: 82.2%
Minibatch loss at step 3000: 0.799750
Minibatch accuracy: 84.4%
Validation accuracy: 82.1%
Test accuracy: 89.1%


agregando regularizacion L2 a una red de una capa oculta

In [7]:
batch_size = 128
num_hidden_nodes=1024
beta = 0.01
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases = tf.Variable(tf.zeros([num_hidden_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  a1 =  tf.matmul(tf_train_dataset, weights) + biases
  z1= tf.nn.relu(a1)
  logits = tf.matmul(z1,weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # regulizer
  regularizers=tf.nn.l2_loss(weights)+tf.nn.l2_loss(weights2)
  loss=tf.reduce_mean(loss+beta*regularizers)  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  #valid
  layer_1_valid=tf.nn.relu(tf.matmul(tf_valid_dataset,weights)+biases)
  logits_valid= tf.matmul(layer_1_valid,weights2)+biases2
    
  valid_prediction = tf.nn.softmax(logits_valid)
  # prediction
  layer_1_prediction=tf.nn.relu(tf.matmul(tf_test_dataset,weights)+biases)
  logits_prediction= tf.matmul(layer_1_prediction,weights2)+biases2 
    
  test_prediction = tf.nn.softmax(logits_prediction)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3454.959473
Minibatch accuracy: 4.7%
Validation accuracy: 18.9%
Minibatch loss at step 500: 1155.199951
Minibatch accuracy: 84.4%
Validation accuracy: 80.2%
Minibatch loss at step 1000: 423.314209
Minibatch accuracy: 76.6%
Validation accuracy: 82.3%
Minibatch loss at step 1500: 154.590302
Minibatch accuracy: 93.0%
Validation accuracy: 84.1%
Minibatch loss at step 2000: 57.003849
Minibatch accuracy: 93.0%
Validation accuracy: 85.5%
Minibatch loss at step 2500: 21.406366
Minibatch accuracy: 89.8%
Validation accuracy: 85.4%
Minibatch loss at step 3000: 8.365267
Minibatch accuracy: 84.4%
Validation accuracy: 84.7%
Test accuracy: 91.8%


# Problem 2

In [9]:
batch_size = 500
num_hidden_nodes=1024
beta = 0.01
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases = tf.Variable(tf.zeros([num_hidden_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  a1 =  tf.matmul(tf_train_dataset, weights) + biases
  z1= tf.nn.relu(a1)
  logits = tf.matmul(z1,weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # regulizer
  regularizers=tf.nn.l2_loss(weights)+tf.nn.l2_loss(weights2)
  loss=tf.reduce_mean(loss+beta*regularizers)  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  #valid
  layer_1_valid=tf.nn.relu(tf.matmul(tf_valid_dataset,weights)+biases)
  logits_valid= tf.matmul(layer_1_valid,weights2)+biases2
    
  valid_prediction = tf.nn.softmax(logits_valid)
  # prediction
  layer_1_prediction=tf.nn.relu(tf.matmul(tf_test_dataset,weights)+biases)
  logits_prediction= tf.matmul(layer_1_prediction,weights2)+biases2 
    
  test_prediction = tf.nn.softmax(logits_prediction)

In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3412.276855
Minibatch accuracy: 12.6%
Validation accuracy: 38.0%
Minibatch loss at step 500: 21.306583
Minibatch accuracy: 84.4%
Validation accuracy: 84.5%
Minibatch loss at step 1000: 0.878468
Minibatch accuracy: 85.4%
Validation accuracy: 83.9%
Minibatch loss at step 1500: 0.744348
Minibatch accuracy: 83.2%
Validation accuracy: 84.4%
Minibatch loss at step 2000: 0.644834
Minibatch accuracy: 85.6%
Validation accuracy: 83.7%
Minibatch loss at step 2500: 0.662709
Minibatch accuracy: 86.4%
Validation accuracy: 84.4%
Minibatch loss at step 3000: 0.706967
Minibatch accuracy: 84.2%
Validation accuracy: 84.0%
Test accuracy: 90.4%


# Problem 3
Dropout

In [11]:
batch_size = 128
num_hidden_nodes=1024
beta = 0.01
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases = tf.Variable(tf.zeros([num_hidden_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  a1 =  tf.matmul(tf_train_dataset, weights) + biases
  z1= tf.nn.relu(a1)
  
  # dropout
  keep_prob=tf.placeholder("float")
  z1_dropout=tf.nn.dropout(z1,keep_prob)

    
  logits = tf.matmul(z1_dropout,weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # regulizer
  regularizers=tf.nn.l2_loss(weights)+tf.nn.l2_loss(weights2)
  loss=tf.reduce_mean(loss+beta*regularizers)  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  #valid
  layer_1_valid=tf.nn.relu(tf.matmul(tf_valid_dataset,weights)+biases)
  logits_valid= tf.matmul(layer_1_valid,weights2)+biases2
    
  valid_prediction = tf.nn.softmax(logits_valid)
  # prediction
  layer_1_prediction=tf.nn.relu(tf.matmul(tf_test_dataset,weights)+biases)
  logits_prediction= tf.matmul(layer_1_prediction,weights2)+biases2 
    
  test_prediction = tf.nn.softmax(logits_prediction)

In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keep_prob:0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3659.748779
Minibatch accuracy: 10.9%
Validation accuracy: 32.5%
Minibatch loss at step 500: 21.366491
Minibatch accuracy: 84.4%
Validation accuracy: 83.7%
Minibatch loss at step 1000: 1.020952
Minibatch accuracy: 78.9%
Validation accuracy: 82.8%
Minibatch loss at step 1500: 0.672247
Minibatch accuracy: 82.0%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 0.693305
Minibatch accuracy: 87.5%
Validation accuracy: 83.0%
Minibatch loss at step 2500: 0.780571
Minibatch accuracy: 82.8%
Validation accuracy: 82.9%
Minibatch loss at step 3000: 0.853155
Minibatch accuracy: 82.0%
Validation accuracy: 83.1%
Test accuracy: 90.0%


# Problem 4
 red de mayor numero de capas

## 2 capas


In [48]:
import math

In [49]:
batch_size = 128
num_hidden_nodes=1024
num_hidden_nodes2=int(num_hidden_nodes*0.5)



beta = 0.001




graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  
    
    
  # Variables.
    # layer 1  (728*1024)
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes], stddev=math.sqrt(2.0/(image_size*image_size))))
  biases = tf.Variable(tf.zeros([num_hidden_nodes]))
    #layer 2 (1024*512)
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_hidden_nodes2], stddev=math.sqrt(2.0/num_hidden_nodes)))
  biases2 = tf.Variable(tf.zeros([num_hidden_nodes2]))
   #layer 3  (512*10)
  weights3 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes2, num_labels],stddev=math.sqrt(2.0/num_hidden_nodes2)))
  biases3 = tf.Variable(tf.zeros([num_labels]))





  # Training computation.
    #layer1
  a1 =tf.matmul(tf_train_dataset,weights)+biases
  z1= tf.nn.relu(a1)
  
  # dropout
  keep_prob=tf.placeholder("float")
  z1_dropout=tf.nn.dropout(z1,keep_prob)
    
 #layer 2
  a2=tf.matmul(z1_dropout,weights2)+biases2
  z2=tf.nn.relu(a2)
  z2_dropout=tf.nn.dropout(z2,keep_prob)  


    
    
    
#layer output    
  logits = tf.matmul(z2_dropout,weights3) + biases3
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # regulizer
  regularizers=tf.nn.l2_loss(weights)+tf.nn.l2_loss(weights2)+tf.nn.l2_loss(weights3)
  loss=tf.reduce_mean(loss+beta*regularizers)  
  # Optimizer.
  # agregando un learning rate descendiente
  global_step=tf.Variable(0)
  start_learning_rate=0.5
  learning_rate=tf.train.exponential_decay(start_learning_rate,global_step,100000,0.96,staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  #valid
  layer_1_valid=tf.nn.relu(tf.matmul(tf_valid_dataset,weights)+biases)
  layer_2_valid=tf.nn.relu(tf.matmul(layer_1_valid,weights2)+biases2) 
  

  logits_valid= tf.matmul(layer_2_valid,weights3)+biases3
    
  valid_prediction = tf.nn.softmax(logits_valid)
  # prediction
  layer_1_prediction=tf.nn.relu(tf.matmul(tf_test_dataset,weights)+biases)
  layer_2_prediction=tf.nn.relu(tf.matmul(layer_1_prediction,weights2)+biases2)
    
  logits_prediction= tf.matmul(layer_2_prediction,weights3)+biases3 
    
  test_prediction = tf.nn.softmax(logits_prediction)

In [50]:
num_steps =15000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keep_prob:0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.684428
Minibatch accuracy: 14.1%
Validation accuracy: 17.1%
Minibatch loss at step 500: 1.232363
Minibatch accuracy: 87.5%
Validation accuracy: 84.5%
Minibatch loss at step 1000: 1.132977
Minibatch accuracy: 82.0%
Validation accuracy: 85.5%
Minibatch loss at step 1500: 0.752156
Minibatch accuracy: 88.3%
Validation accuracy: 86.3%
Minibatch loss at step 2000: 0.681600
Minibatch accuracy: 92.2%
Validation accuracy: 86.6%
Minibatch loss at step 2500: 0.653703
Minibatch accuracy: 87.5%
Validation accuracy: 86.7%
Minibatch loss at step 3000: 0.650823
Minibatch accuracy: 85.9%
Validation accuracy: 86.8%
Minibatch loss at step 3500: 0.719965
Minibatch accuracy: 83.6%
Validation accuracy: 86.8%
Minibatch loss at step 4000: 0.551236
Minibatch accuracy: 86.7%
Validation accuracy: 86.9%
Minibatch loss at step 4500: 0.549462
Minibatch accuracy: 88.3%
Validation accuracy: 86.7%
Minibatch loss at step 5000: 0.587611
Minibatch accuracy: 87.5%
Validation accurac

# 5 capas

In [51]:
batch_size = 128
num_hidden_nodes=1024
num_hidden_nodes2=int(num_hidden_nodes*0.5)
num_hidden_nodes3=int(num_hidden_nodes*np.power(0.5, 2))
num_hidden_nodes4=int(num_hidden_nodes*np.power(0.5, 3))
num_hidden_nodes5=int(num_hidden_nodes*np.power(0.5, 4))
num_hidden_nodes6=int(num_hidden_nodes*np.power(0.5, 5))



beta = 0.001




graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  
    
    
  # Variables.
    # layer 1  (728*1024)
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes],stddev=math.sqrt(2.0/( image_size * image_size))))
  biases = tf.Variable(tf.zeros([num_hidden_nodes]))    #layer 2 (1024*512)
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_hidden_nodes2],stddev=math.sqrt(2.0/num_hidden_nodes)))
  biases2 = tf.Variable(tf.zeros([num_hidden_nodes2]))
   #layer 3  (512*256)
  weights3 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes2, num_hidden_nodes3],stddev=math.sqrt(2.0/num_hidden_nodes2)))
  biases3 = tf.Variable(tf.zeros([num_hidden_nodes3]))

   #layer 4 (256*128)
  weights4 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes3, num_hidden_nodes4],stddev=math.sqrt(2.0/num_hidden_nodes3)))
  biases4 = tf.Variable(tf.zeros([num_hidden_nodes4]))

   #layer 5 (128*64)
  weights5 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes4, num_hidden_nodes5],stddev=math.sqrt(2.0/num_hidden_nodes4)))
  biases5 = tf.Variable(tf.zeros([num_hidden_nodes5]))

   #layer 6 (64*10)
  weights6 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes5, num_labels],stddev=math.sqrt(2.0/num_hidden_nodes5)))
  biases6 = tf.Variable(tf.zeros([num_labels]))




  # Training computation.
    #layer1
  a1 =tf.matmul(tf_train_dataset,weights)+biases
  z1= tf.nn.relu(a1)
  
  # dropout
  keep_prob=tf.placeholder("float")
  z1_dropout=tf.nn.dropout(z1,keep_prob)
    
 #layer 2
  a2=tf.matmul(z1_dropout,weights2)+biases2
  z2=tf.nn.relu(a2)
  z2_dropout=tf.nn.dropout(z2,keep_prob)  

 #layer 3
  a3=tf.matmul(z2_dropout,weights3)+biases3
  z3=tf.nn.relu(a3)
  z3_dropout=tf.nn.dropout(z3,keep_prob)  

    
 #layer 4
  a4=tf.matmul(z3_dropout,weights4)+biases4
  z4=tf.nn.relu(a4)
  z4_dropout=tf.nn.dropout(z4,keep_prob)  

 #layer 4
  a5=tf.matmul(z4_dropout,weights5)+biases5
  z5=tf.nn.relu(a5)
  z5_dropout=tf.nn.dropout(z5,keep_prob)  
    
    
    
    
#layer output    
  logits = tf.matmul(z5_dropout,weights6) + biases6
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # regulizer
  regularizers=tf.nn.l2_loss(weights)+tf.nn.l2_loss(weights2)+tf.nn.l2_loss(weights3)+tf.nn.l2_loss(weights4)+tf.nn.l2_loss(weights5)+tf.nn.l2_loss(weights6)
  loss=tf.reduce_mean(loss+beta*regularizers)  
  # Optimizer.
  # agregando un learning rate descendiente
  global_step=tf.Variable(0)
  start_learning_rate=0.5
  learning_rate=tf.train.exponential_decay(start_learning_rate,global_step,100000,0.96,staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  #valid
  layer_1_valid=tf.nn.relu(tf.matmul(tf_valid_dataset,weights)+biases)
  layer_2_valid=tf.nn.relu(tf.matmul(layer_1_valid,weights2)+biases2) 
  layer_3_valid=tf.nn.relu(tf.matmul(layer_2_valid,weights3)+biases3)  
  layer_4_valid=tf.nn.relu(tf.matmul(layer_3_valid,weights4)+biases4)
  layer_5_valid=tf.nn.relu(tf.matmul(layer_4_valid,weights5)+biases5)
  

  logits_valid= tf.matmul(layer_5_valid,weights6)+biases6
    
  valid_prediction = tf.nn.softmax(logits_valid)
  # prediction
  layer_1_prediction=tf.nn.relu(tf.matmul(tf_test_dataset,weights)+biases)
  layer_2_prediction=tf.nn.relu(tf.matmul(layer_1_prediction,weights2)+biases2)
  layer_3_prediction=tf.nn.relu(tf.matmul(layer_2_prediction,weights3)+biases3)
  layer_4_prediction=tf.nn.relu(tf.matmul(layer_3_prediction,weights4)+biases4)
  layer_5_prediction=tf.nn.relu(tf.matmul(layer_4_prediction,weights5)+biases5)
    
  logits_prediction= tf.matmul(layer_5_prediction,weights6)+biases6 
    
  test_prediction = tf.nn.softmax(logits_prediction)

In [52]:
num_steps = 15000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keep_prob:0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.030939
Minibatch accuracy: 8.6%
Validation accuracy: 10.8%
Minibatch loss at step 500: 1.713111
Minibatch accuracy: 83.6%
Validation accuracy: 82.9%
Minibatch loss at step 1000: 1.403155
Minibatch accuracy: 82.0%
Validation accuracy: 83.4%
Minibatch loss at step 1500: 1.107686
Minibatch accuracy: 85.2%
Validation accuracy: 84.4%
Minibatch loss at step 2000: 0.927455
Minibatch accuracy: 88.3%
Validation accuracy: 85.3%
Minibatch loss at step 2500: 0.940555
Minibatch accuracy: 85.9%
Validation accuracy: 84.3%
Minibatch loss at step 3000: 0.824810
Minibatch accuracy: 85.2%
Validation accuracy: 85.2%
Minibatch loss at step 3500: 0.816075
Minibatch accuracy: 82.0%
Validation accuracy: 85.3%
Minibatch loss at step 4000: 0.796557
Minibatch accuracy: 85.9%
Validation accuracy: 85.8%
Minibatch loss at step 4500: 0.794048
Minibatch accuracy: 83.6%
Validation accuracy: 85.5%
Minibatch loss at step 5000: 0.796185
Minibatch accuracy: 86.7%
Validation accuracy